In [ ]:
import os
from PIL import Image
import numpy as np

In [ ]:
image_path = "C:/Data/DeepFashion/images/"
mask_path = "C:/Data/DeepFashion/segm/segm/"

In [ ]:
images = os.listdir(image_path)
masks = os.listdir(mask_path)

In [ ]:
len(images),len(masks)

In [ ]:
images = []
for i in masks:
    images.append(i.replace("_segm.png", ".jpg"))
len(images)

In [ ]:
for i, j in zip(images, masks):
    img = np.array(Image.open(os.path.join(image_path, i)).convert("RGB"))
    mask = np.array(Image.open(os.path.join(mask_path, j)).convert("RGB"))
    break

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.show()
plt.imshow(mask)
plt.show()

In [ ]:
mask.shape

In [ ]:
from UNet import UNet
import torch

In [ ]:
img = np.resize(img, (256, 128))

In [ ]:
img = torch.from_numpy(img)
x = img.type(torch.float32)

In [ ]:
x.shape

In [ ]:
x = x.reshape((1,3, 256, 128))

In [ ]:
x = model(x)

In [ ]:
x.shape

In [ ]:
x = x.reshape((1101, 750, 3))
x = x.detach().numpy()

In [ ]:
plt.imshow((x * 255).astype(np.uint8))

In [ ]:
x.shape

In [1]:
import torch
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from UNet import UNet
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loader,
)

#HyperParameters
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2
NUM_EPOCHS = 10
NUM_WORKERS = 2
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 128
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = "C:/Data/DeepFashion/images/"
TRAIN_MASK_DIR = "C:/Data/DeepFashion/segm/segm/"

In [2]:
# train_transform = A.Compose(
#     [
# #         A.Normalize(
# #             mean=[0.0, 0.0, 0.0],
# #             std = [1.0, 1.0, 1.0],
# #             max_pixel_value=255.0,
# #         ),
#         ToTensorV2(),
#     ]
# )

train_loader = get_loader(TRAIN_IMG_DIR, TRAIN_MASK_DIR, BATCH_SIZE)

In [3]:
train_loader

In [4]:
model = torch.load("C:/Users/Jaitej/OneDrive/Desktop/Virtual Try On 2.0/my_checkpoint.pth.tar")

In [5]:
with torch.inference_mode():
    for image, mask in train_loader:
        image = image.to(DEVICE)
        image = image.float()
        x = model(image)
        img = mask.float()
        break

In [7]:
image.shape, mask.shape

(torch.Size([2, 3, 512, 256]), torch.Size([2, 24, 512, 256]))

In [8]:
img = img.reshape(2, 24, 512, 256)
x = x.reshape(2, 24, 512, 256)
image = image.reshape(2, 512, 256, 3)


img = img.cpu()
x = x.cpu()
image = image.cpu()

In [9]:
import matplotlib.pyplot as plt
from PIL import Image

In [10]:
def decode(a):
    return torch.argmax(a, dim=0)

def decode_mask(a):
    probs = torch.softmax(a, dim=0)
    preds = torch.argmax(probs, dim=0)
    return preds

In [11]:
fig, ax = plt.subplots(nrows=16, ncols=3, figsize=(100,500))
for i in range(2):
    im = img[i]
    mask = x[i]
    
    ax[i][0].imshow(image[i])
    ax[i][1].imshow(decode(im))
    ax[i][2].imshow(decode_mask(mask))
    


In [ ]:
torch.max(mask)

In [ ]:
mask

In [ ]:
import torchvision.transforms as T

transform = T.PILToTensor()

In [ ]:
f = "C:/Data/DeepFashion/segm/segm/MEN-Denim-id_00000080-01_7_additional_segm.png"

from PIL import Image
import numpy as np

segm = Image.open(f)
segm = segm.resize((128, 256))
a = transform(segm) # shap

In [ ]:
a

In [ ]:
a = a[0]

In [ ]:
mask = torch.zeros((24, 256, 128))

In [ ]:
mask.shape[0]

In [ ]:
for i in range(mask.shape[0]):
    mask[i] = torch.as_tensor((a == i), dtype=torch.int8) 

In [ ]:
torch.max(mask)

In [ ]:
plt.imshow(a)

In [ ]:
a = a.resize(1101, 750)

In [ ]:
b.shape

In [ ]:
plt.imshow(b)

In [ ]:
torch.max(b)

In [ ]:
torch.max(a)